# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

/opt/hostedtoolcache/Python/3.8.16/x64/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7f4d10e85d90>
├── 'a' --> tensor([[ 0.6016,  1.7956,  0.0658],
│                   [-0.4611,  0.8411, -1.0774]])
└── 'x' --> <FastTreeValue 0x7f4d10e85d60>
    └── 'c' --> tensor([[ 1.2556,  0.4568, -2.2765,  0.0121],
                        [-0.4745,  0.7363,  0.3724,  0.1834],
                        [ 0.3052, -1.2309, -0.0143, -1.0517]])

In [4]:
t.a

tensor([[ 0.6016,  1.7956,  0.0658],
        [-0.4611,  0.8411, -1.0774]])

In [5]:
%timeit t.a

62 ns ± 0.0414 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7f4d10e85d90>
├── 'a' --> tensor([[ 0.6549, -1.4419,  0.9126],
│                   [ 0.1614,  0.8798, -1.8105]])
└── 'x' --> <FastTreeValue 0x7f4d10e85d60>
    └── 'c' --> tensor([[ 1.2556,  0.4568, -2.2765,  0.0121],
                        [-0.4745,  0.7363,  0.3724,  0.1834],
                        [ 0.3052, -1.2309, -0.0143, -1.0517]])

In [7]:
%timeit t.a = new_value

59.3 ns ± 0.0758 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 0.6016,  1.7956,  0.0658],
               [-0.4611,  0.8411, -1.0774]]),
    x: Batch(
           c: tensor([[ 1.2556,  0.4568, -2.2765,  0.0121],
                      [-0.4745,  0.7363,  0.3724,  0.1834],
                      [ 0.3052, -1.2309, -0.0143, -1.0517]]),
       ),
)

In [10]:
b.a

tensor([[ 0.6016,  1.7956,  0.0658],
        [-0.4611,  0.8411, -1.0774]])

In [11]:
%timeit b.a

51.1 ns ± 0.04 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[-2.6083,  0.2126,  0.0657],
               [ 1.1948, -0.2392, -1.0506]]),
    x: Batch(
           c: tensor([[ 1.2556,  0.4568, -2.2765,  0.0121],
                      [-0.4745,  0.7363,  0.3724,  0.1834],
                      [ 0.3052, -1.2309, -0.0143, -1.0517]]),
       ),
)

In [13]:
%timeit b.a = new_value

486 ns ± 0.151 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

851 ns ± 0.147 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.3 µs ± 27.7 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

137 µs ± 1.32 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

137 µs ± 305 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7f4d10e858b0>
├── 'a' --> tensor([[[ 0.6016,  1.7956,  0.0658],
│                    [-0.4611,  0.8411, -1.0774]],
│           
│                   [[ 0.6016,  1.7956,  0.0658],
│                    [-0.4611,  0.8411, -1.0774]],
│           
│                   [[ 0.6016,  1.7956,  0.0658],
│                    [-0.4611,  0.8411, -1.0774]],
│           
│                   [[ 0.6016,  1.7956,  0.0658],
│                    [-0.4611,  0.8411, -1.0774]],
│           
│                   [[ 0.6016,  1.7956,  0.0658],
│                    [-0.4611,  0.8411, -1.0774]],
│           
│                   [[ 0.6016,  1.7956,  0.0658],
│                    [-0.4611,  0.8411, -1.0774]],
│           
│                   [[ 0.6016,  1.7956,  0.0658],
│                    [-0.4611,  0.8411, -1.0774]],
│           
│                   [[ 0.6016,  1.7956,  0.0658],
│                    [-0.4611,  0.8411, -1.0774]]])
└── 'x' --> <FastTreeValue 0x7f4d10e85ac0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

32 µs ± 57.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7f4d10e710a0>
├── 'a' --> tensor([[ 0.6016,  1.7956,  0.0658],
│                   [-0.4611,  0.8411, -1.0774],
│                   [ 0.6016,  1.7956,  0.0658],
│                   [-0.4611,  0.8411, -1.0774],
│                   [ 0.6016,  1.7956,  0.0658],
│                   [-0.4611,  0.8411, -1.0774],
│                   [ 0.6016,  1.7956,  0.0658],
│                   [-0.4611,  0.8411, -1.0774],
│                   [ 0.6016,  1.7956,  0.0658],
│                   [-0.4611,  0.8411, -1.0774],
│                   [ 0.6016,  1.7956,  0.0658],
│                   [-0.4611,  0.8411, -1.0774],
│                   [ 0.6016,  1.7956,  0.0658],
│                   [-0.4611,  0.8411, -1.0774],
│                   [ 0.6016,  1.7956,  0.0658],
│                   [-0.4611,  0.8411, -1.0774]])
└── 'x' --> <FastTreeValue 0x7f4c64b807f0>
    └── 'c' --> tensor([[ 1.2556,  0.4568, -2.2765,  0.0121],
                        [-0.4745,  0.7363,  0.3724,  0.1834],
                 

In [23]:
%timeit t_cat(trees)

30 µs ± 74.5 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

56.8 µs ± 95.8 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    a: tensor([[[ 0.6016,  1.7956,  0.0658],
                [-0.4611,  0.8411, -1.0774]],
       
               [[ 0.6016,  1.7956,  0.0658],
                [-0.4611,  0.8411, -1.0774]],
       
               [[ 0.6016,  1.7956,  0.0658],
                [-0.4611,  0.8411, -1.0774]],
       
               [[ 0.6016,  1.7956,  0.0658],
                [-0.4611,  0.8411, -1.0774]],
       
               [[ 0.6016,  1.7956,  0.0658],
                [-0.4611,  0.8411, -1.0774]],
       
               [[ 0.6016,  1.7956,  0.0658],
                [-0.4611,  0.8411, -1.0774]],
       
               [[ 0.6016,  1.7956,  0.0658],
                [-0.4611,  0.8411, -1.0774]],
       
               [[ 0.6016,  1.7956,  0.0658],
                [-0.4611,  0.8411, -1.0774]]]),
    x: Batch(
           c: tensor([[[ 1.2556,  0.4568, -2.2765,  0.0121],
                       [-0.4745,  0.7363,  0.3724,  0.1834],
                       [ 0.3052, -1.2309, -0.0143, -1.0517]],
         

In [26]:
%timeit Batch.stack(batches)

77.7 µs ± 135 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    a: tensor([[ 0.6016,  1.7956,  0.0658],
               [-0.4611,  0.8411, -1.0774],
               [ 0.6016,  1.7956,  0.0658],
               [-0.4611,  0.8411, -1.0774],
               [ 0.6016,  1.7956,  0.0658],
               [-0.4611,  0.8411, -1.0774],
               [ 0.6016,  1.7956,  0.0658],
               [-0.4611,  0.8411, -1.0774],
               [ 0.6016,  1.7956,  0.0658],
               [-0.4611,  0.8411, -1.0774],
               [ 0.6016,  1.7956,  0.0658],
               [-0.4611,  0.8411, -1.0774],
               [ 0.6016,  1.7956,  0.0658],
               [-0.4611,  0.8411, -1.0774],
               [ 0.6016,  1.7956,  0.0658],
               [-0.4611,  0.8411, -1.0774]]),
    x: Batch(
           c: tensor([[ 1.2556,  0.4568, -2.2765,  0.0121],
                      [-0.4745,  0.7363,  0.3724,  0.1834],
                      [ 0.3052, -1.2309, -0.0143, -1.0517],
                      [ 1.2556,  0.4568, -2.2765,  0.0121],
                      [-0.4745,  

In [28]:
%timeit Batch.cat(batches)

144 µs ± 423 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

314 µs ± 2.57 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
